In [18]:
import openpyxl
import requests_random_user_agent
import requests
import pandas as pd 
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urlunparse
import re
import csv
import os

class GetFinancialData:
    def __init__(self, cik_file_path, filing_type):
        self.cik_file_path = cik_file_path
        self.filing_type = filing_type
        self.cik_data = {}
        self.EDGAR_search_results = {}
        self.filing_data = {}
        self.financial_information = {}
        self.base_url_sec = r"https://www.sec.gov"

    def read_cik_data(self):
        with open(self.cik_file_path, 'r') as file:
            lines = file.readlines()

        for line in lines:
            line = line.strip()
            parts = line.split()

            if len(parts) == 2:
                word = parts[0]
                cik = parts[1]
                self.cik_data[word] = cik

    def fetch_filings(self):
        endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

        for company, cik in self.cik_data.items():
            param_dict = {'action':'getcompany',
                          'CIK': cik,
                          'type': self.filing_type,
                          'owner':'exclude',
                          'output':'',
                          'count':'100'}

            response = requests.get(url=endpoint, params=param_dict)
            soup = BeautifulSoup(response.content, 'html.parser')

            if response.status_code == 200:
                print('Request Successful')
                print(response.url)
                self.EDGAR_search_results[company] = response
            else:
                print('Request Failed')

    def get_filing_data(self):
        for company, url in self.EDGAR_search_results.items():
            liste = f"filing_{company}"
            soup = BeautifulSoup(url.content, 'html.parser')
            doc_table = soup.find_all('table', class_='tableFile2')
            master_list = []

            for row in doc_table[0].find_all('tr'):
                cols = row.find_all('td')

                if len(cols) != 0:
                    filing_type = cols[0].text.strip()
                    filing_date = cols[3].text.strip()
                    filing_numb = cols[4].text.strip()
                    filing_doc_href = cols[1].find('a', {'href':True, 'id':'documentsbutton'})
                    filing_int_href = cols[1].find('a', {'href':True, 'id':'interactiveDataBtn'})
                    filing_num_href = cols[4].find('a')

                    if filing_doc_href != None:
                        filing_doc_link = self.base_url_sec + filing_doc_href['href'] 
                    else:
                        filing_doc_link = 'no link'

                    if filing_int_href != None:
                        filing_int_link = self.base_url_sec + filing_int_href['href'] 
                    else:
                        filing_int_link = 'no link'

                    if filing_num_href != None:
                        filing_num_link = self.base_url_sec + filing_num_href['href'] 
                    else:
                        filing_num_link = 'no link'

                    file_dict = {}
                    file_dict['file_type'] = filing_type
                    file_dict['file_number'] = filing_numb
                    file_dict['file_date'] = filing_date
                    file_dict['links'] = {}
                    file_dict['links']['documents'] = filing_doc_link

                    document_response = requests.get(filing_doc_link)
                    document_soup = BeautifulSoup(document_response.content, 'html.parser')
                    document_links = document_soup.find_all('a', {'href': lambda href: href.endswith('.txt')})
                    document = [document['href'] for document in document_links]
                    full_txt_url = "https://www.sec.gov" + document[0]
                    pattern = r"/\d{12,20}/"
                    modified_url = re.sub(pattern, "/",full_txt_url)
                    document_url = modified_url.replace('-','').replace('.txt', '/index.json')
                    base_url = r"https://www.sec.gov"

                    try:
                        content = requests.get(document_url).json()
                        for file in content['directory']['item']:

                            if file['name'] == 'FilingSummary.xml':
                                xml_summary = base_url + content['directory']['name']  + "/" + file['name']
                                file_dict['links']['documents_xml'] = xml_summary

                    except requests.exceptions.RequestException as e:
                        print("Error occurred for:", document_url)
                        continue

                    file_dict['links']['interactive_data'] = filing_int_link
                    file_dict['links']['filing_number'] = filing_num_link
                    master_list.append(file_dict)

                self.filing_data[f"filing_{company}"] = master_list

    def process_financial_information(self):
        for i in self.filing_data.keys():
            big_liste = {}
            for j in range(0, len(self.filing_data[i])):
                documents_xml = self.filing_data[i][j]['links'].get('documents_xml')
                try:
                    xml_summary = documents_xml
                    base_url = xml_summary.replace('FilingSummary.xml', '')
                    content = requests.get(xml_summary).content
                    soup = BeautifulSoup(content, 'lxml')
                    reports = soup.find('myreports')
                    master_reports = []

                    try:
                        for report in reports.find_all('report')[:-1]:
                            date = report['instance'].replace('.htm', '').replace('.xml', '')
                            report_dict = {}
                            report_dict[date] = {}
                            report_dict[date]['name_short'] = report.shortname.text
                            report_dict[date]['name_long'] = report.longname.text
                            report_dict[date]['position'] = report.position.text
                            report_dict[date]['category'] = report.menucategory.text
                            report_dict[date]['url'] = base_url + report.htmlfilename.text
                            master_reports.append(report_dict)

                        big_liste[f"{date}"] = master_reports

                    except Exception as e:
                        content = requests.get(documents_xml).content
                        soup = BeautifulSoup(content, 'lxml')
                        try:
                            InputFiles = soup.find('inputfiles')
                            date = soup.find('file').text.replace('.xml', '').replace('.htm','')
                            for report in reports.find_all('report')[:-1]:
                                report_dict = {}
                                report_dict[date] = {}
                                report_dict[date]['name_short'] = report.shortname.text
                                report_dict[date]['name_long'] = report.longname.text
                                report_dict[date]['url'] = base_url + report.htmlfilename.text
                                master_reports.append(report_dict)

                            big_liste[f"{date}"] = master_reports
                        except Exception as e:
                            print("error occured for:", documents_xml)
                except Exception as e:
                    print('no xml document for:', self.filing_data[i][j])

            self.financial_information[f"{i}"] = big_liste

if __name__ == "__main__":
    cik_file_path = "cik/liste_cik.txt"
    filing_type = "10-k"

    data_processor = GetFinancialData(cik_file_path, filing_type)
    data_processor.read_cik_data()
    data_processor.fetch_filings()
    data_processor.get_filing_data()
    data_processor.process_financial_information()
    financial_information = data_processor.financial_information

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=320193&type=10-k&owner=exclude&output=&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=789019&type=10-k&owner=exclude&output=&count=100
Error occurred for: https://www.sec.gov/Archives/edgar/data/320193/a2032880z10k/index.json
Error occurred for: https://www.sec.gov/Archives/edgar/data/789019/dex32/index.json
Error occurred for: https://www.sec.gov/Archives/edgar/data/789019/d10k/index.json
Error occurred for: https://www.sec.gov/Archives/edgar/data/789019/0001032210000019610001/index.json


/Users/gabry/opt/anaconda3/envs/freqtrade_V1/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


error occured for: https://www.sec.gov/Archives/edgar/data/320193/000119312510238044/FilingSummary.xml
error occured for: https://www.sec.gov/Archives/edgar/data/320193/000119312510012091/FilingSummary.xml
error occured for: https://www.sec.gov/Archives/edgar/data/320193/000119312509214859/FilingSummary.xml
no xml document for: {'file_type': '10-K', 'file_number': '000-10030081162315', 'file_date': '2008-11-05', 'links': {'documents': 'https://www.sec.gov/Archives/edgar/data/320193/000119312508224958/0001193125-08-224958-index.htm', 'interactive_data': 'no link', 'filing_number': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=000-10030&owner=exclude&count=100'}}
no xml document for: {'file_type': '10-K', 'file_number': '000-10030071250316', 'file_date': '2007-11-15', 'links': {'documents': 'https://www.sec.gov/Archives/edgar/data/320193/000104746907009340/0001047469-07-009340-index.htm', 'interactive_data': 'no link', 'filing_number': 'https://www.sec.gov/cgi-bin/b

In [6]:
# Save files locally, 
folder_name = "financial_information"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

def create_excel_files(data):

    for filing_name, filing_data in data.items():
        with pd.ExcelWriter(os.path.join(folder_name, f'{filing_name}.xlsx')) as writer:
            for key, records in filing_data.items():
                df = pd.DataFrame([record[key] for record in records])
                df.to_excel(writer, sheet_name=key, index=False)

create_excel_files(financial_information)

### Use case for '10-k' filings

In [35]:
def process_financial_statements(financial_information, report_list):
    filing_url = {}
    tables = {}
    final_product = {}

    for filing_name in financial_information.keys():
        for filing_date in financial_information[filing_name].keys():
            statements_url = []
            for i in range(0, len(financial_information[filing_name][filing_date])):
                name_short = financial_information[filing_name][filing_date][i][filing_date]['name_short']
                if any(re.search(item, name_short.lower()) for item in report_list):
                    print('-' * 100)
                    print(name_short)
                    print(financial_information[filing_name][filing_date][i][filing_date]['url'])
                    statements_url.append(financial_information[filing_name][filing_date][i][filing_date]['url'])
            filing_url[f'url_{filing_date}'] = statements_url

    for date in filing_url:
        statements_data = []
        for url in filing_url[date]:
            statement_data = {}
            statement_data['headers'] = []
            statement_data['sections'] = []
            statement_data['data'] = []

            content = requests.get(url).content
            report_soup = BeautifulSoup(content, 'html')

            for index, row in enumerate(report_soup.table.find_all('tr')):
                cols = row.find_all('td')

                if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0):
                    reg_row = [ele.text.strip() for ele in cols]
                    statement_data['data'].append(reg_row)

                elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
                    sec_row = cols[0].text.strip()
                    statement_data['sections'].append(sec_row)

                elif (len(row.find_all('th')) != 0):
                    hed_row = [ele.text.strip() for ele in row.find_all('th')]
                    statement_data['headers'].append(hed_row)

                else:
                    print('We encountered an error.')

            statements_data.append(statement_data)
            tables[f'{date}'] = statements_data

    for key in tables.keys():
        final_product[f'{key}'] = {}
        for i in range(0, len(tables[key])):
            try:
                data_type = tables[key][i]['headers'][0][0]
                headers = tables[key][i]['headers'][0][1:]
                data = tables[key][i]['data']
                data = [item for item in data if item]

                df = pd.DataFrame(data)

                df.index = df[0]
                df.index.name = 'Category'
                df = df.drop(0, axis=1)

                df = df.replace('[\$,)]', '', regex=True) \
                    .replace('[(]', '-', regex=True) \
                    .replace('', 'NaN', regex=True)

                df = df.apply(pd.to_numeric, errors='coerce')

                non_nan_count = df.count()

                for k in range(1, 10):
                    selected_columns = non_nan_count[non_nan_count > k].index
                    if len(selected_columns) == len(headers):
                        break

                df = df[selected_columns]
                df.columns = headers

                final_product[f'{key}'][f'{data_type}'] = df

            except Exception as e:

                try:

                    data_type = tables[key][i]['headers'][0][0]
                    headers = tables[key][i]['headers'][0]
                    data = tables[key][i]['data']

                    df = pd.DataFrame(data)

                    df.index = df[0]
                    df.index.name = 'Category'
                    df = df.drop(0, axis=1)

                    df = df.replace('[\$,)]', '', regex=True) \
                        .replace('[(]', '-', regex=True) \
                        .replace('', 'NaN', regex=True)

                    df = df.apply(pd.to_numeric, errors='coerce')

                    non_nan_count = df.count()

                    for k in range(1, 10):
                        selected_columns = non_nan_count[non_nan_count > k].index
                        if len(selected_columns) == len(headers):
                            break

                    df = df[selected_columns]
                    df.columns = headers
                    final_product[f'{key}'][f'{data_type}'] = df

                except Exception as e:

                    print('error occurred for:', key)

    return final_product

# Example usage

# Define the statements we want to look for.
# item must be lower case and can be incomplete

#item = r"balance sheets"
item = r"consolidated statements of operations"
#item = r"Consolidated Statements of Cash Flows"
#item = r"Consolidated Statements of Stockholder"
report_list = [item]
processed_data = process_financial_statements(financial_information, report_list)


----------------------------------------------------------------------------------------------------
CONSOLIDATED STATEMENTS OF OPERATIONS
https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/R3.htm
----------------------------------------------------------------------------------------------------
Segment Information and Geographic Data - Reconciliation of Segment Operating Income to the Consolidated Statements of Operations (Details)
https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/R65.htm
----------------------------------------------------------------------------------------------------
CONSOLIDATED STATEMENTS OF OPERATIONS
https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/R2.htm
----------------------------------------------------------------------------------------------------
Segment Information and Geographic Data - Reconciliation of Segment Operating Income to the Consolidated Statements of Operations (Details)
https://www.sec.gov/

## Example usage

### Define the statements we want to look for.
### Item must be lower case and can be incomplete

In [ ]:
#item = r"balance sheets"
item = r"consolidated statements of operations"
#item = r"Consolidated Statements of Cash Flows"
#item = r"Consolidated Statements of Stockholder"
report_list = [item]
processed_data = process_financial_statements(financial_information, report_list)

In [39]:
processed_data['url_aapl-20160924']['Comprehensive Income - Pre-tax Amounts Reclassified from AOCI into Consolidated Statements of Operations (Detail) - USD ($) $ in Millions']

,3 Months Ended,12 Months Ended
Category,,
Revenue,-215639.0,-233715.0
Cost of sales,131376.0,140089.0
"Other income/(expense), net",1348.0,1285.0
Income before provision for income taxes,-61372.0,-72515.0
Reclassification out of Accumulated Other Comprehensive Income,NaN,NaN
Income before provision for income taxes,-785.0,-4036.0
Reclassification out of Accumulated Other Comprehensive Income | Unrealized Gains/Losses on Derivative Instruments,NaN,NaN
Income before provision for income taxes,-872.0,-4127.0
Reclassification out of Accumulated Other Comprehensive Income | Unrealized Gains/Losses on Derivative Instruments | Foreign exchange contracts,NaN,NaN
